Advanced Features
---

By now, you’ve learned the most commonly used parts of the Rust programming  
language. Before we do one more project in Chapter 20, we’ll look at a few  
aspects of the language you might run into every once in a while, but may not  
use every day. You can use this chapter as a reference for when you encounter  
any unknowns. The features covered here are useful in very specific situations.  
Although you might not reach for them often, we want to make sure you have a  
grasp of all the features Rust has to offer.

In this chapter, we’ll cover:

- **`Unsafe Rust:`** how to opt out of some of Rust’s guarantees and take  
responsibility for manually upholding those guarantees
- **`Advanced traits:`** associated types, default type parameters, fully  
qualified syntax, supertraits, and the newtype pattern in relation to traits
- **`Advanced types:`** more about the newtype pattern, type aliases, the never  
type, and dynamically sized types
- **`Advanced functions and closures:`** function pointers and returning  
closures  
- **`Macros:`** ways to define code that defines more code at compile time  

It’s a panoply of Rust features with something for everyone! Let’s dive in!

### Unsafe Rust

All the code we’ve discussed so far has had Rust’s memory safety guarantees  
enforced at compile time. However, Rust has a second language hidden inside it  
that doesn’t enforce these memory safety guarantees:  

- it’s called unsafe Rust and works just like regular Rust, but gives us extra  
superpowers.

**`Unsafe Rust`** exists because, by nature, static analysis is conservative.  
When the compiler tries to determine whether or not code upholds the guarantees,  
it’s better for it to reject some valid programs than to accept some invalid  
programs. Although the code might be okay, if the Rust compiler doesn’t have  
enough information to be confident, it will reject the code. In these cases,  
you can use unsafe code to tell the compiler, “Trust me, I know what I’m doing.”  
Be warned, however, that you use unsafe Rust at your own risk:  

- if you use unsafe code incorrectly, problems can occur due to memory unsafety,  
such as null pointer dereferencing.

Another reason Rust has an unsafe alter ego is that the underlying computer  
hardware is inherently unsafe. If Rust didn’t let you do unsafe operations, you  
couldn’t do certain tasks. Rust needs to allow you to do low-level systems  
programming, such as directly interacting with the operating system or even  
writing your own operating system. Working with low-level systems programming is  
one of the goals of the language. Let’s explore what we can do with unsafe Rust  
and how to do it.

### Unsafe SuperPowers

To switch to unsafe Rust, use the **`unsafe`** keyword and then start a new  
block that holds the unsafe code. You can take five actions in unsafe Rust that  
you can’t in safe Rust, which we call unsafe superpowers. Those superpowers  
include the ability to:

- Dereference a raw pointer
- Call an unsafe function or method
- Access or modify a mutable static variable
- Implement an unsafe trait
- Access fields of `union`s

It’s important to understand that unsafe doesn’t turn off the borrow checker or  
disable any other of Rust’s safety checks:  
- if you use a reference in unsafe code, it will still be checked.  
The unsafe keyword only gives you access to these five features that are then  
not checked by the compiler for memory safety. You’ll still get some degree of  
safety inside of an unsafe block.

In addition, unsafe does not mean the code inside the block is necessarily  
dangerous or that it will definitely have memory safety problems:  

- the intent is that as the programmer, you’ll ensure the code inside an unsafe  
block will access memory in a valid way.

People are fallible, and mistakes will happen, but by requiring these five  
unsafe operations to be inside blocks annotated with unsafe you’ll know that  
any errors related to memory safety must be within an unsafe block. Keep unsafe  
blocks small; you’ll be thankful later when you investigate memory bugs.

To isolate unsafe code as much as possible, it’s best to enclose unsafe code  
within a safe abstraction and provide a safe API, which we’ll discuss later in  
the chapter when we examine unsafe functions and methods. Parts of the standard  
library are implemented as safe abstractions over unsafe code that has been  
audited. Wrapping unsafe code in a safe abstraction prevents uses of unsafe  
from leaking out into all the places that you or your users might want to use  
the functionality implemented with unsafe code, because using a safe abstraction  
is safe.

Let’s look at each of the five unsafe superpowers in turn. We’ll also look at  
some abstractions that provide a safe interface to unsafe code.

### Dereferencing a Raw Pointer

In Chapter 4, in the “Dangling References” section, we mentioned that the  
compiler ensures references are always valid. Unsafe Rust has two new types  
called raw pointers that are similar to references. As with references, raw  
pointers can be immutable or mutable and are written as *const T and *mut T,  
respectively. The asterisk isn’t the dereference operator; it’s part of the  
type name. In the context of raw pointers, immutable means that the pointer  
can’t be directly assigned to after being dereferenced.

Different from references and smart pointers, raw pointers:

- Are allowed to ignore the borrowing rules by having both immutable and  
mutable pointers or multiple mutable pointers to the same location
- Aren’t guaranteed to point to valid memory
- Are allowed to be null
- Don’t implement any automatic cleanup

By opting out of having Rust enforce these guarantees, you can give up  
guaranteed safety in exchange for greater performance or the ability to  
interface with another language or hardware where Rust’s guarantees don’t  
apply.

> Listing 19-1 shows how to create an immutable and a mutable raw pointer  
from references.

In [2]:
{
    let mut num = 5;

    let r1 = &num as *const i32;
    let r2 = &mut num as *mut i32;
}

()

> Listing 19-1: Creating raw pointers from references

Notice that we don’t include the unsafe keyword in this code. We can create raw  
pointers in safe code; we just can’t dereference raw pointers outside an unsafe  
block, as you’ll see in a bit.

We’ve created raw pointers by using as to cast an immutable and a mutable  
reference into their corresponding raw pointer types. Because we created them  
directly from references guaranteed to be valid, we know these particular raw  
pointers are valid, but we can’t make that assumption about just any raw pointer.

To demonstrate this, next we’ll create a raw pointer whose validity we can’t be  
so certain of. Listing 19-2 shows how to create a raw pointer to an arbitrary  
location in memory. Trying to use arbitrary memory is undefined: there might be  
data at that address or there might not, the compiler might optimize the code  
so there is no memory access, or the program might error with a segmentation  
fault. Usually, there is no good reason to write code like this, but it is  
possible.

In [3]:
{
    let address = 0x012345usize;
    let r = address as *const i32;
}

()

> Listing 19-2: Creating a raw pointer to an arbitrary memory address

Recall that we can create raw pointers in safe code, but we can’t dereference  
raw pointers and read the data being pointed to. In Listing 19-3, we use the  
dereference operator * on a raw pointer that requires an unsafe block.

In [6]:
{
    let mut num = 5;

    let r1 = &num as *const i32;
    let r2 = &mut num as *mut i32;

    unsafe {
        // Creating a pointer does no harm @audit : why???
        // It's only when we try to access the value that it points at that we
        // might end up dealing with an invalid value
        println!("r1 is {}", *r1);
        println!("r2 is {}", *r2);
    }
}

r1 is 5
r2 is 5


()

> Listing 19-3: Dereferencing raw pointers within an unsafe block

Creating a pointer does no harm; it’s only when we try to access the value that  
it points at that we might end up dealing with an invalid value.

Note also that in Listing 19-1 and 19-3, we created *const i32 and *mut i32 raw  
pointers that both pointed to the same memory location, where num is stored. If  
we instead tried to create an immutable and a mutable reference to num, the code  
would not have compiled because Rust’s ownership rules don’t allow a mutable  
reference at the same time as any immutable references. With raw pointers, we  
can create a mutable pointer and an immutable pointer to the same location and  
change data through the mutable pointer, potentially creating a data race.  
Be careful!

With all of these dangers, why would you ever use raw pointers? One major use  
case is when interfacing with C code, as you’ll see in the next section,  
“Calling an Unsafe Function or Method.” Another case is when building up safe  
abstractions that the borrow checker doesn’t understand. We’ll introduce unsafe  
functions and then look at an example of a safe abstraction that uses unsafe  
code.

### Calling an Unsafe Function or Method

The second type of operation you can perform in an unsafe block is calling  
unsafe functions. Unsafe functions and methods look exactly like regular  
functions and methods, but they have an extra unsafe before the rest of the  
definition. The unsafe keyword in this context indicates the function has  
requirements we need to uphold when we call this function, because Rust can’t  
guarantee we’ve met these requirements. By calling an unsafe function within an  
unsafe block, we’re saying that we’ve read this function’s documentation and  
take responsibility for upholding the function’s contracts.

Here is an unsafe function named dangerous that doesn’t do anything in its  
body:

In [13]:
unsafe fn dangerous() {}

// this unsafe block is required to call an unsafe function
unsafe {
    dangerous();
}

()

We must call the dangerous function within a separate unsafe block. If we try  
to call dangerous without the unsafe block, we’ll get an error:

```sh
$ cargo run
   Compiling unsafe-example v0.1.0 (file:///projects/unsafe-example)
error[E0133]: call to unsafe function is unsafe and requires unsafe function or block
 --> src/main.rs:4:5
  |
4 |     dangerous();
  |     ^^^^^^^^^^^ call to unsafe function
  |
  = note: consult the function's documentation for information on how to avoid undefined behavior

For more information about this error, try `rustc --explain E0133`.
error: could not compile `unsafe-example` due to previous error
```

With the unsafe block, we’re asserting to Rust that we’ve read the function’s  
documentation, we understand how to use it properly, and we’ve verified that  
we’re fulfilling the contract of the function.

Bodies of unsafe functions are effectively unsafe blocks, so to perform other  
unsafe operations within an unsafe function, we don’t need to add another  
unsafe block.

### Creating a Safe Abstraction over Unsafe Code

Just because a function contains unsafe code doesn’t mean we need to mark the  
entire function as unsafe. In fact, wrapping unsafe code in a safe function is  
a common abstraction. As an example, let’s study the split_at_mut function from  
the standard library, which requires some unsafe code. We’ll explore how we  
might implement it. 

This safe method is defined on mutable slices:  
- it takes one slice and makes it two by splitting the slice at the index given  
as an argument. 

> Listing 19-4 shows how to use split_at_mut.

In [20]:
{
    let mut v = vec![1, 2, 3, 4, 5, 6];

    let r = &mut v[..];

    let (a, b) = r.split_at_mut(3);

    assert_eq!(a, &mut [1, 2, 3]);
    assert_eq!(b, &mut [4, 5, 6]);
}

()

> Listing 19-4: Using the safe split_at_mut function

We can’t implement this function using only safe Rust. An attempt might look  
something like Listing 19-5, which won’t compile. For simplicity, we’ll  
implement split_at_mut as a function rather than a method and only for slices  
of i32 values rather than for a generic type T.

In [35]:
/// Takes a slice
/// - split at mid
/// - returns a tuple of the resulting 2 slices
fn split_at_mut(values: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]){
    let len = values.len();

    // safety check that `mid` is within our slice bounds
    // - else, PANIC and TERMINATE program HERE
    assert!(mid <= len);

    // @audit-ok : rust borrow checker will not allow this because `values` is 
    // mutable
    // - and can ONLY be borrowed ONCE
    // - in generating the 2nd slice, `values` is borrowed AGAIN
    (&mut values[..mid], &mut values[mid..])
    // Borrowing different NON OVERLAPPING parts of a slice is FUNDAMENTALLY OK
    // @audit-ok ... but Rust borrow checker DOES NOT KNOW THIS
    // - this is a SUFFICIENT ENOUGH basis to deploy `unsafe` code
}

Error: cannot borrow `*values` as mutable more than once at a time

> Listing 19-5: An attempted implementation of split_at_mut using only safe Rust

This function first gets the total length of the slice. Then it asserts that  
the index given as a parameter is within the slice by checking whether it’s  
less than or equal to the length. The assertion means that if we pass an index  
that is greater than the length to split the slice at, the function will panic  
before it attempts to use that index.

Then we return two mutable slices in a tuple: 

- one from the start of the original slice to the mid index  
- and another from mid to the end of the slice.

When we try to compile the code in Listing 19-5, we’ll get an error.

```sh
$ cargo run
   Compiling unsafe-example v0.1.0 (file:///projects/unsafe-example)
error[E0499]: cannot borrow `*values` as mutable more than once at a time
 --> src/main.rs:6:31
  |
1 | fn split_at_mut(values: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]) {
  |                         - let's call the lifetime of this reference `'1`
...
6 |     (&mut values[..mid], &mut values[mid..])
  |     --------------------------^^^^^^--------
  |     |     |                   |
  |     |     |                   second mutable borrow occurs here
  |     |     first mutable borrow occurs here
  |     returning this value requires that `*values` is borrowed for `'1`

For more information about this error, try `rustc --explain E0499`.
error: could not compile `unsafe-example` due to previous error
```

Rust’s borrow checker can’t understand that we’re borrowing different parts of  
the slice; it only knows that we’re borrowing from the same slice twice.  

- Borrowing different parts of a slice is fundamentally okay  
- because the two slices aren’t overlapping,  
- but Rust isn’t smart enough to know this.  

When we know code is okay, but Rust doesn’t, it’s time to reach for unsafe code.

Listing 19-6 shows how to use an unsafe block, a raw pointer, and some calls to  
unsafe functions to make the implementation of split_at_mut work.

In [32]:
// import slice module from standard library
// - provides functions for handling slices
use std::slice;

/// Takes a slice : 
/// - splits at mid
/// - returns a tuple of those two slices
fn split_at_mut(value: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]){

    // get original slice length
    let len = value.len();
    // get RAW POINTER to the data in the slice
    // - RAW POINTER allow for MORE CONTROL and LESS SAFETY in Rust
    let ptr = value.as_mut_ptr();

    // safety check that mid is within the slice bound
    // - if out of bound, PANIC and TERMINATE the program
    assert!(mid <= len);

    // start an unsafe block
    unsafe {
        // returns a tuple of two mutable slices
        (
            // from_raw_parts_mut : creates a mutable slice from :
            // - ptr : a raw pointer to the first element in the slice
            // - the length of the slice : mid
            slice::from_raw_parts_mut(ptr, mid),
            // from_raw_parts_mut : creates a mutable slice from :
            // - ptr : a raw pointer to starting point of ([0] + mid)
            // - the length of the slice : (len - mid)
            slice::from_raw_parts_mut(ptr.add(mid), len -mid)
        )
    }
}

> Listing 19-6: Using unsafe code in the implementation of the split_at_mut  
function

Recall from “The Slice Type” section in Chapter 4 that slices are a pointer to  
some data and the length of the slice. We use the len method to get the length  
of a slice and the as_mut_ptr method to access the raw pointer of a slice. In  
this case, because we have a mutable slice to i32 values, as_mut_ptr returns a  
raw pointer with the type *mut i32, which we’ve stored in the variable ptr.

We keep the assertion that the mid index is within the slice. Then we get to the  
unsafe code: the slice::from_raw_parts_mut function takes a raw pointer and a  
length, and it creates a slice. We use this function to create a slice that  
starts from ptr and is mid items long. Then we call the add method on ptr with  
mid as an argument to get a raw pointer that starts at mid, and we create a  
slice using that pointer and the remaining number of items after mid as the  
length.

The function slice::from_raw_parts_mut is unsafe because it takes a raw pointer  
and must trust that this pointer is valid. The add method on raw pointers is  
also unsafe, because it must trust that the offset location is also a valid  
pointer. Therefore, we had to put an unsafe block around our calls to  
slice::from_raw_parts_mut and add so we could call them. By looking at the code  
and by adding the assertion that mid must be less than or equal to len, we can  
tell that all the raw pointers used within the unsafe block will be valid  
pointers to data within the slice. This is an acceptable and appropriate use of  
unsafe.

Note that we don’t need to mark the resulting split_at_mut function as unsafe,  
and we can call this function from safe Rust. We’ve created a safe abstraction  
to the unsafe code with an implementation of the function that uses unsafe code  
in a safe way, because it creates only valid pointers from the data this  
function has access to.

In contrast, the use of slice::from_raw_parts_mut in Listing 19-7 would likely  
crash when the slice is used. This code takes an arbitrary memory location and  
creates a slice 10,000 items long.

In [10]:
{
    use std::slice;

    let address = 0x01234usize;
    // @audit-ok : Why is the address printed as 0x1234 => 4660
    // - how is the address 0x01234usize => 4660
    // ANSWER :
    // 0*16^4 + 1*16^3 + 2*16^2 + 3*16^1 + 4*16^0
    // =    0 + 4096   + 512    + 48     + 4 = 4660
    println!("address dec : {}", address);
    println!("address hex : {:x}", address);
    let r = address as *mut i32;

    let values: &[i32] = unsafe {
        slice::from_raw_parts_mut(r, 10000)
    };

    // trying to print the value causes a segfault
    // - Segmentation fault.
    // 0: backtrace::capture::Backtrace::new
    // 1: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
    // 2: __platform_memmove
    // ...
    println!("{:?}", values);
}

address dec : 4660
address hex : 1234


Segmentation fault.
   0: backtrace::capture::Backtrace::new
   1: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   3: core::fmt::builders::DebugInner::entry
   4: core::fmt::builders::DebugInner::entry
   5: core::fmt::builders::DebugList::entry
   6: <&T as core::fmt::Debug>::fmt
   7: core::fmt::write
   2: __platform_memmove
   8: <&std::io::stdio::Stdout as std::io::Write>::write_fmt
  10: std::panicking::try
   9: std::io::stdio::_print
  11: _run_user_code_6
  12: evcxr::runtime::Runtime::run_loop
  13: evcxr::runtime::runtime_hook
  14: evcxr_jupyter::main
  15: std::sys_common::backtrace::__rust_begin_short_backtrace
  16: std::rt::lang_start::{{closure}}
  17: std::rt::lang_start_internal
  18: _main



Error: Subprocess terminated with status: signal: 6 (SIGABRT)

> Listing 19-7: Creating a slice from an arbitrary memory location

We don’t own the memory at this arbitrary location, and there is no guarantee  
that the slice this code creates contains valid i32 values. Attempting to use  
values as though it’s a valid slice results in undefined behavior.

### Using `extern` Functions to Call External Code

Sometimes, your Rust code might need to interact with code written in another  
language. For this, Rust has the keyword extern that facilitates the creation  
and use of a Foreign Function Interface (FFI). An FFI is a way for a programming  
language to define functions and enable a different (foreign) programming  
language to call those functions.

Listing 19-8 demonstrates how to set up an integration with the abs function  
from the C standard library. Functions declared within extern blocks are always  
unsafe to call from Rust code. The reason is that other languages don’t enforce  
Rust’s rules and guarantees, and Rust can’t check them, so responsibility falls  
on the programmer to ensure safety.

> Filename: src/main.rs

In [6]:
{
    // Jupyter cell errors if we try to run this cell multiple times
    // - [E0428] Error: the name `abs` is defined multiple times
    // - so we will enclose this in a cell block
    extern "C" {
        // abs is an actual C function???
        // - @audit : How do I get a list of other available C functions?
        fn abs(input: i32)->i32;
    }

    unsafe {
        println!("Absolute value of -3 according to C: {}", abs(-3));
    }

}

Absolute value of -3 according to C: 3


()

> Listing 19-8: Declaring and calling an extern function defined in another  
language

Within the extern "C" block, we list the names and signatures of external  
functions from another language we want to call. The "C" part defines which  
application binary interface (ABI) the external function uses: the ABI defines  
how to call the function at the assembly level. The "C" ABI is the most common  
and follows the C programming language’s ABI.

#### -- Calling Rust Functions from Other Languages --

We can also use extern to create an interface that allows other languages to  
call Rust functions. Instead of creating a whole extern block, we add the  
extern keyword and specify the ABI to use just before the fn keyword for the  
relevant function. We also need to add a #[no_mangle] annotation to tell the  
Rust compiler not to mangle the name of this function. Mangling is when a  
compiler changes the name we’ve given a function to a different name that  
contains more information for other parts of the compilation process to consume  
but is less human readable. Every programming language compiler mangles names  
slightly differently, so for a Rust function to be nameable by other languages,  
we must disable the Rust compiler’s name mangling.

In the following example, we make the call_from_c function accessible from C  
code, after it’s compiled to a shared library and linked from C:

```rust
#[no_mangle]
pub extern "C" fn call_from_c() {
    println!("Just called a Rust function from C!");
}
```

> This usage of extern does not require unsafe.


In [10]:

#[no_mangle]
pub extern "C" fn call_from_c() {
    println!("Just called a Rust function from C!");
}

### Accessing or Modifying a Mutable Static Variable

In this book, we’ve not yet talked about global variables, which Rust does  
support but can be problematic with Rust’s ownership rules. If two threads are  
accessing the same mutable global variable, it can cause a data race.

In Rust, global variables are called static variables. Listing 19-9 shows an  
example declaration and use of a static variable with a string slice as a value.

> Filename: src/main.rs

In [11]:
static HELLO_WORLD: &str = "Hello, world!";

{
    println!("name is : {}", HELLO_WORLD);
}

name is : Hello, world!


()

> Listing 19-9: Defining and using an immutable static variable

Static variables are similar to constants, which we discussed in the  
“Differences Between Variables and Constants” section in Chapter 3. The names  
of static variables are in SCREAMING_SNAKE_CASE by convention. Static variables  
can only store references with the 'static lifetime, which means the Rust  
compiler can figure out the lifetime and we aren’t required to annotate it  
explicitly. Accessing an immutable static variable is safe.

A subtle difference between constants and immutable static variables is that :  
- values in a static variable have a fixed address in memory.  
- Using the value will always access the same data. 
- Constants, on the other hand, are allowed to duplicate their data whenever  
they’re used.  
- Another difference is that static variables can be mutable.  

Accessing and modifying mutable static variables is unsafe. Listing 19-10 shows  
how to declare, access, and modify a mutable static variable named COUNTER.

> Filename: src/main.rs

In [14]:
static mut COUNTER: u32 = 0;

fn add_to_count(inc: u32) {
    unsafe {
        COUNTER += inc;
    }
}

{
    add_to_count(3);

    unsafe {
        println!("COUNTER : {}", COUNTER);
    }
}

COUNTER : 3


()

> Listing 19-10: Reading from or writing to a mutable static variable is unsafe

As with regular variables, we specify mutability using the mut keyword. Any  
code that :  
- reads or writes from COUNTER 
- must be within an unsafe block.  

This code compiles and prints COUNTER: 3 as we would expect because it’s single  
threaded. Having multiple threads access COUNTER would likely result in data  
races.

With mutable data that is globally accessible, it’s difficult to ensure there  
are no data races, which is why Rust considers mutable static variables to be  
unsafe. Where possible, it’s preferable to use the concurrency techniques and  
thread-safe smart pointers we discussed in Chapter 16 so the compiler checks  
that data accessed from different threads is done safely.

### Implementing an Unsafe Trait

We can use unsafe to implement an unsafe trait. A trait is unsafe when :  

- at least one of its methods has some invariant that the compiler can’t verify.  

We declare that a trait is unsafe by :  
- adding the unsafe keyword before trait and  
- marking the implementation of the trait as unsafe too, 

> as shown in Listing 19-11.



In [15]:
unsafe trait Foo {
    // methods go here
}

unsafe impl Foo for i32 {
    // method implementations go here
}

{
    // main calls here
}

()

> Listing 19-11: Defining and implementing an unsafe trait

By using unsafe impl, we’re promising that we’ll uphold the invariants that the  
compiler can’t verify.

As an example, recall the Sync and Send marker traits we discussed in the  
“Extensible Concurrency with the Sync and Send Traits” section in Chapter 16:  
the compiler implements these traits automatically if our types are composed  
entirely of Send and Sync types. If we implement a type that contains a type  
that is not Send or Sync, such as raw pointers, and we want to mark that type  
as Send or Sync, we must use unsafe. Rust can’t verify that our type upholds  
the guarantees that it can be safely sent across threads or accessed from  
multiple threads; therefore, we need to do those checks manually and indicate  
as such with unsafe.

### Accessing Fields of a Union

The final action that works only with unsafe is accessing fields of a union. A  
union is similar to a struct, but only one declared field is used in a  
particular instance at one time. Unions are primarily used to interface with  
unions in C code. Accessing union fields is unsafe because Rust can’t guarantee  
the type of the data currently being stored in the union instance. You can  
learn more about unions in the Rust Reference.

### When to Use Unsafe Code

Using unsafe to take one of the five actions (superpowers) just discussed isn’t  
wrong or even frowned upon. But it is trickier to get unsafe code correct  
because the compiler can’t help uphold memory safety. When you have a reason to  
use unsafe code, you can do so, and having the explicit unsafe annotation makes  
it easier to track down the source of problems when they occur.

Advanced Traits
---

We first covered traits in the “Traits: Defining Shared Behavior” section of  
Chapter 10, but we didn’t discuss the more advanced details. Now that you know  
more about Rust, we can get into the nitty-gritty.

### Specifying PlaceHolder Types in Trait Definitions with Associated Types

Associated types connect a type placeholder with a trait such that the trait  
method definitions can use these placeholder types in their signatures. The  
implementor of a trait will specify the concrete type to be used instead of the  
placeholder type for the particular implementation. That way, we can define a  
trait that uses some types without needing to know exactly what those types are  
until the trait is implemented.

We’ve described most of the advanced features in this chapter as being rarely  
needed. Associated types are somewhere in the middle: they’re used more rarely  
than features explained in the rest of the book but more commonly than many of  
the other features discussed in this chapter.

One example of a trait with an associated type is the Iterator trait that the  
standard library provides. The associated type is named Item and stands in for  
the type of the values the type implementing the Iterator trait is iterating  
over. The definition of the Iterator trait is as shown in Listing 19-12.

In [4]:
pub trait Iterator {
    type Item;

    fn next(&mut self) -> Option<Self::Item>;
}

> Listing 19-12: The definition of the Iterator trait that has an associated type  
Item

The type Item is a placeholder, and the next method’s definition shows that it  
will return values of type Option<Self::Item>. Implementors of the Iterator  
trait will specify the concrete type for Item, and the next method will return  
an Option containing a value of that concrete type.

Associated types might seem like a similar concept to generics, in that the  
latter allow us to define a function without specifying what types it can  
handle. To examine the difference between the two concepts, we’ll look at an  
implementation of the Iterator trait on a type named Counter that specifies the  
Item type is u32:

> Filename: src/lib.rs

In [5]:
struct Counter {}

impl Iterator for Counter {
    type Item = u32;

    fn next(&mut self) -> Option<Self::Item> {
        // --snip--
        None
    }
}

This syntax seems comparable to that of generics. So why not just define the  
Iterator trait with generics, as shown in  

> Listing 19-13?

In [6]:
pub trait Iterator<T> {
    fn next(&mut self) -> Option<T>;
}

Error: type `Item` is not a member of trait `Iterator`

Error: missing generics for trait `Iterator`

Error: associated type `Item` not found for `Self`

> Listing 19-13: A hypothetical definition of the Iterator trait using generics

The difference is that when using generics, as in Listing 19-13, we must  
annotate the types in each implementation; because we can also implement  
Iterator<String> for Counter or any other type, we could have multiple  
implementations of Iterator for Counter. In other words, when a trait has a  
generic parameter, it can be implemented for a type multiple times, changing  
the concrete types of the generic type parameters each time. When we use the  
next method on Counter, we would have to provide type annotations to indicate  
which implementation of Iterator we want to use.

With associated types, we don’t need to annotate types because we can’t  
implement a trait on a type multiple times. In Listing 19-12 with the  
definition that uses associated types, we can only choose what the type of  
Item will be once, because there can only be one impl Iterator for Counter. We  
don’t have to specify that we want an iterator of u32 values everywhere that we  
call next on Counter.

Associated types also become part of the trait’s contract:  
- implementors of the trait must provide a type to stand in for the associated  
type placeholder. Associated types often have a name that describes how the  
type will be used, and documenting the associated type in the API documentation  
is good practice.

### Default Generic Type Parameters and Operator Overloading

When we use generic type parameters, we can specify a default concrete type for  
the generic type. This eliminates the need for implementors of the trait to  
specify a concrete type if the default type works. You specify a default type  
when declaring a generic type with the <PlaceholderType=ConcreteType> syntax.

A great example of a situation where this technique is useful is with operator  
overloading, in which you customize the behavior of an operator (such as +) in  
particular situations.

Rust doesn’t allow you to create your own operators or overload arbitrary  
operators. But you can overload the operations and corresponding traits listed  
in std::ops by implementing the traits associated with the operator. For  
example, in Listing 19-14 we overload the + operator to add two Point instances  
together. We do this by implementing the Add trait on a Point struct:

> Filename: src/main.rs

In [7]:
use std::ops::Add;

#[derive(Debug, Copy, Clone, PartialEq)]
struct Point {
    x: i32,
    y: i32,
}

impl Add for Point {
    type Output = Point;

    fn add(self, other: Point) -> Point {
        Point {
            x: self.x + other.x,
            y: self.y + other.y,
        }
    }
}

{
    assert_eq! {
        Point { x: 1, y: 0 } + Point { x: 2, y: 3 },
        Point { x: 3, y: 3 }
    }
}

()

> Listing 19-14: Implementing the Add trait to overload the + operator for  
Point instances

The add method :  

- adds the x values of two Point instances  
- and the y values of two Point instances  
- to create a new Point.  

The Add trait has an associated type named Output that determines the type  
returned from the add method.

The default generic type in this code is within the Add trait. Here is its  
definition:

```rust
trait Add<Rhs=Self> {
    type Output;

    fn add(self, rhs: Rhs) -> Self::Output;
}
```

This code should look generally familiar: a trait with one method and an  
associated type. The new part is Rhs=Self: this syntax is called default  
type parameters. The Rhs generic type parameter (short for “right hand side”)  
defines the type of the rhs parameter in the add method. If we don’t specify a  
concrete type for Rhs when we implement the Add trait, the type of Rhs will  
default to Self, which will be the type we’re implementing Add on.

When we implemented Add for Point, we used the default for Rhs because we  
wanted to add two Point instances. Let’s look at an example of implementing the  
Add trait where we want to customize the Rhs type rather than using the default.

We have two structs, Millimeters and Meters, holding values in different units.  
This thin wrapping of an existing type in another struct is known as the  
newtype pattern, which we describe in more detail in the “Using the Newtype  
Pattern to Implement External Traits on External Types” section. We want to add  
values in millimeters to values in meters and have the implementation of Add do  
the conversion correctly. We can implement Add for Millimeters with Meters as  
the Rhs, as shown in Listing 19-15.

> Filename: src/lib.rs

In [9]:
use std::ops::Add;

struct Millimeters(u32);
struct Meters(u32);

// impl block for the Add trait for : 
// - the Millimeters struct
// - when adding a Meters struct
impl Add<Meters> for Millimeters {
    // we want to return millimeters ...
    type Output = Millimeters;

    // when adding other(Meters) to self(Millimeters)
    fn add(self, other: Meters) -> Millimeters {
        // we return a new Millimeters struct
        Millimeters(self.0 + (other.0 * 1000)) // 1 meter = 1000 millimeters
    }
}

> Listing 19-15: Implementing the Add trait on Millimeters to add Millimeters to  
Meters

To add Millimeters and Meters, we specify impl Add<Meters> to set the value of  
the Rhs type parameter instead of using the default of Self.

You’ll use default type parameters in two main ways:

- To extend a type without breaking existing code
- To allow customization in specific cases most users won’t need

The standard library’s Add trait is an example of the second purpose:  

- usually, you’ll add two like types,  
- but the Add trait provides the ability to customize beyond that.  

Using a default type parameter in the Add trait definition means you don’t have  
to specify the extra parameter most of the time. In other words, a bit of  
implementation boilerplate isn’t needed, making it easier to use the trait.

The first purpose is similar to the second but in reverse: if you want to add a  
type parameter to an existing trait, you can give it a default to allow  
extension of the functionality of the trait without breaking the existing  
implementation code.

### Fully Qualified Syntax for Disambiguation: Calling Methods with the Same Name

Nothing in Rust prevents a trait from having a method with the same name as  
another trait’s method, nor does Rust prevent you from implementing both traits  
on one type. It’s also possible to implement a method directly on the type with  
the same name as methods from traits.

When calling methods with the same name, you’ll need to tell Rust which one you  
want to use. Consider the code in Listing 19-16 where we’ve defined two traits,  
Pilot and Wizard, that both have a method called fly. We then implement both  
traits on a type Human that already has a method named fly implemented on it.  
Each fly method does something different.

> Filename: src/main.rs

In [19]:
trait Pilot {
    fn fly(&self);
}

trait Wizard {
    fn fly(&self);
}

struct Human;

impl Pilot for Human {
    fn fly(&self) {
        println!("This is your captain speaking.");
    }
}

impl Wizard for Human {
    fn fly(&self) {
        println!("Up!");
    }
}

impl Human {
    fn fly(&self) {
        println!("*waving arms furiously*");
    }
}


> Listing 19-16: Two traits are defined to have a fly method and are implemented  
on the Human type, and a fly method is implemented on Human directly

When we call fly on an instance of Human, the compiler defaults to calling the  
method that is directly implemented on the type, as shown in Listing 19-17.

> Filename: src/main.rs

In [20]:
{
    let person = Human;
    person.fly();
}

*waving arms furiously*


()

> Listing 19-17: Calling fly on an instance of Human

Running this code will print *waving arms furiously*, showing that Rust called  
the fly method implemented on Human directly.

To call the fly methods from either the Pilot trait or the Wizard trait, we  
need to use more explicit syntax to specify which fly method we mean.  

> Listing 19-18 demonstrates this syntax.

> Filename: src/main.rs

In [24]:
{
    let person = Human;
    Pilot::fly(&person);
    Wizard::fly(&person);
    person.fly();
}

This is your captain speaking.
Up!
*waving arms furiously*


()

> Listing 19-18: Specifying which trait’s fly method we want to call

Specifying the trait name before the method name clarifies to Rust which  
implementation of fly we want to call. We could also write Human::fly(&person),  
which is equivalent to the person.fly() that we used in Listing 19-18, but this  
is a bit longer to write if we don’t need to disambiguate.

Running this code prints the following:

```sh
$ cargo run
   Compiling traits-example v0.1.0 (file:///projects/traits-example)
    Finished dev [unoptimized + debuginfo] target(s) in 0.46s
     Running `target/debug/traits-example`
This is your captain speaking.
Up!
*waving arms furiously*
```

Because the fly method takes a self parameter, if we had two types that both  
implement one trait, Rust could figure out which implementation of a trait to  
use based on the type of self.

However, associated functions that are not methods don’t have a self parameter.  
When there are multiple types or traits that define non-method functions with  
the same function name, Rust doesn't always know which type you mean unless you  
use fully qualified syntax. For example, in Listing 19-19 we create a trait for  
an animal shelter that wants to name all baby dogs Spot. We make an Animal trait  
with an associated non-method function baby_name. The Animal trait is  
implemented for the struct Dog, on which we also provide an associated  
non-method function baby_name directly.

> Filename: src/main.rs